In [ ]:
from tqdm import tqdm
from queue import Queue
import cv2 as cv
import inaccel.coral as inaccel
import io
import ipywidgets as widgets
import numpy as np
import PIL.Image
import threading
import time

### Settings

In [ ]:
FILTER_SIZE = 3
req_num = 8
queue_size = 16 # not necessary, but it helps with resource management
scale_down = 16 # must be greater than 7
input_video = 'data/bbb_sunflower_1080p_30fps_stereo_abl.mp4'

### Create ndarrays for the constant parameters of the accelerator
to help the scheduling we create one copy of each array for each request

In [ ]:
#1# Create ndarrays for the constant parameters of the accelerator
Low_thresh = []
High_thresh = []
Shape = []
for i in range(req_num):
    # Create vectors holding thresholds and shape:
    high_thresh = inaccel.ndarray((FILTER_SIZE * FILTER_SIZE), dtype = np.uint8)
    low_thresh = inaccel.ndarray((FILTER_SIZE * FILTER_SIZE), dtype = np.uint8)
    shape = inaccel.ndarray((FILTER_SIZE * FILTER_SIZE), dtype = np.uint8)

    for i in range(FILTER_SIZE*FILTER_SIZE):
        shape[i] = 1

    # Define the low and high thresholds
    # Want to grab 3 colors (Yellow, Green, Red) for the input image
    low_thresh[0] = 22 # Lower boundary for Yellow
    low_thresh[1] = 150
    low_thresh[2] = 60

    high_thresh[0] = 38 # Upper boundary for Yellow
    high_thresh[1] = 255
    high_thresh[2] = 255

    low_thresh[3] = 38 # Lower boundary for Green
    low_thresh[4] = 150
    low_thresh[5] = 60

    high_thresh[3] = 75 # Upper boundary for Green
    high_thresh[4] = 255
    high_thresh[5] = 255

    low_thresh[6] = 160 # Lower boundary for Red
    low_thresh[7] = 150
    low_thresh[8] = 60

    high_thresh[6] = 179 # Upper boundary for Red
    high_thresh[7] = 255
    high_thresh[8] = 255
    
    Low_thresh.append(low_thresh)
    High_thresh.append(high_thresh)
    Shape.append(shape)







### This function for each frame submits requests to coral

In [ ]:
#2# Submitter
def submitter():
    for i in range(frame_num):
        # try to read a new frame
        ret, frame = input_stream.read()
        if (ret == False):
            queue.put((None, None, None, None))
            break

        in_img = inaccel.array(frame, dtype = np.uint8)
        # Allocate the memory for output images:
        out_img = inaccel.ndarray((in_img.shape[0],in_img.shape[1]), dtype = np.uint8)
        # describe a new request
        # Create a request:
        request = inaccel.request('com.xilinx.vitis.vision.colordetect')

        # Set request arguments:
        request.arg(in_img)
        request.arg(Low_thresh[i%req_num])
        request.arg(High_thresh[i%req_num])
        request.arg(Shape[i%req_num])
        request.arg(out_img)
        request.arg(np.int32(in_img.shape[0]))
        request.arg(np.int32(in_img.shape[1]))
        # try to submit that request
        try:
            session = inaccel.submit(request)
        except RuntimeError:
            print("Cannot connect to the Docker daemon. Is InAccel Coral running?")

            queue.put((None, None, None, None))
            break
            
        # push the session reference as well as the ndarrays to the queue
        queue.put((session, in_img, out_img, frame))
        
        # update the queue status
        queue_status.value = queue.qsize() / queue_size

### This function waits the complition of the requests and shows the input and output images

In [ ]:
#3# Waiter
def waiter():
    for i in tqdm(range(frame_num)):
        start = time.time() * 1000
        
        # try to pop from the queue
        session, in_img, out_img, frame = queue.get()
        if (session == None):
            break
        
        # convert the array to an image and write it to the widget
        frame_img = PIL.Image.fromarray(frame)
        f = io.BytesIO()
        frame_img.save(f, format = 'jpeg')
        image_in.value = f.getvalue()
        image_in.width = int(width / scale_down)
        image_in.height = int(height / scale_down)
        
        # wait for the session to complete
        inaccel.wait(session)
        
        # normalize the output array
        disp_img = out_img.view(np.ndarray).astype(np.uint8)
        disp_img = cv.resize(cv.cvtColor(disp_img, cv.COLOR_GRAY2BGR), (int(width / scale_down), int(height / scale_down)))

        # convert the array to an image and write it to the widget
        disp_img = PIL.Image.fromarray(disp_img)
        f = io.BytesIO()
        disp_img.save(f, format = 'jpeg')
        image_out.value = f.getvalue()
        image_out.width = int(width / scale_down)
        image_out.height = int(height / scale_down)        
        
        
        # explicitly delete the inaccel ndarrays
        # this step is not necessary, as the will be garbage collected
        # but it helps free earlier the corresponding device memory
        del in_img
        del out_img
        
        # update the queue status
        queue.task_done()

### Create a VideoCapture object and Start Threads

In [ ]:
# Create a VideoCapture object
input_stream = cv.VideoCapture(input_video)
if (input_stream.isOpened() == False): 
      print("Error while opening the input video stream")

# get the input's properties
frame_num = int(input_stream.get(cv.CAP_PROP_FRAME_COUNT))
height = int(input_stream.get(cv.CAP_PROP_FRAME_HEIGHT) / 2)
width  = int(input_stream.get(cv.CAP_PROP_FRAME_WIDTH))
fps = int(input_stream.get(cv.CAP_PROP_FPS))

print("Video Info:")
print("   Frames: " + str(frame_num))
print("   Dimensions: " + str(height * 2) + "x" + str(width))
print("   Framerate: " + str(fps) + " fps")


In [ ]:
# initialize the widgets
image_in = widgets.Image()
image_out = widgets.Image()

queue_status = widgets.FloatProgress(description = 'Queue:', value = 0.0, min = 0.0, max = 1.0)
images = [image_in, image_out]
image_box = widgets.HBox(images)

# initialize the queue
queue = Queue(maxsize = queue_size)

# initialize the two threads
thread1 = threading.Thread(target = waiter)
thread2 = threading.Thread(target = submitter)

display(queue_status)
display(image_box)

# start the threads
thread1.start()
thread2.start()

# wait for the threads to complete
queue.join()
thread1.join()
thread2.join()